In [1]:
import os

In [4]:
pwd

'c:\\mlflow_dvc_cancer_classification'

In [3]:
cd ..

c:\mlflow_dvc_cancer_classification


In [ ]:
#for config.yaml file 

prepare_base_model :    
root_dir: artifacts/prepare_base_model
base_model_path:artifacts/prepare_base_model/base_model.h5
updated_base_model_path:artifacts/prepare_base_model/updated_base_model.h5
    """ params_image_size:224
    params_learning_rate:0.01
    params_include_top:false
    params_weights:'imagenet'
    params_classes:2 """


SyntaxError: invalid syntax (2788121675.py, line 3)

In [ ]:
from dataclasses import dataclass
from pathlib import Path    

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [ ]:
import os
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.entity.config_entity import (DataIngestionConfig)

# -----------------------------------------------------------------------------
# WHY THIS FILE EXISTS:
# This is the "Configuration Manager". 
# 
# In a complex project, we have:
# 1. Settings in YAML files (config.yaml, params.yaml)
# 2. Code that needs those settings.
#
# This file acts as the BRIDGE. It reads the YAML files, converts them into 
# strict "Entity" objects (like DataIngestionConfig), and gives them to the code.
# -----------------------------------------------------------------------------

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        """
        WHAT: The Constructor. It runs automatically when you create a ConfigurationManager.
        
        WHY: 
        - We need to load the YAML files immediately so the settings are ready to use.
        - We also create the main 'artifacts' folder here because everything else 
          will go inside it.
        
        HOW:
        - Uses 'read_yaml' (from our common.py toolbox) to load the files.
        - Uses 'create_directories' to make the root artifacts folder.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """
        WHAT: Returns the PrepareBaseModelConfig from our config.yaml file.
        
        WHY: 
        - The PrepareBaseModelConfig is a special class that holds all the settings
          needed to train a base model (like VGG16/ResNet).
        - It's used by the PrepareBaseModel class to know what to do.
        
        HOW:
        - Reads the 'prepare_base_model' section from config.yaml.
        - Converts it into a PrepareBaseModelConfig object.
        """
        prepare_base_model_config = self.config.prepare_base_model
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(prepare_base_model_config.root_dir),
            base_model_path=Path(prepare_base_model_config.base_model_path),
            updated_base_model_path=Path(prepare_base_model_config.updated_base_model_path),
            params_image_size=prepare_base_model_config.params_image_size,
            params_learning_rate=prepare_base_model_config.params_learning_rate,
            params_include_top=prepare_base_model_config.params_include_top,
            params_weights=prepare_base_model_config.params_weights,
            params_classes=prepare_base_model_config.params_classes,
            params_epochs=prepare_base_model_config.params_epochs,
            params_batch_size=prepare_base_model_config.params_batch_size,
            params_augmentation=prepare_base_model_config.params_augmentation
        )
        return prepare_base_model_config

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        base_model = tf.keras.applications.VGG16(
            weights=self.config.params_weights,
            include_top=self.config.params_include_top,
            input_shape=self.config.params_image_size,
            #classes=self.config.params_classes
        )

        self.save_model(path=self.config.updated_base_model_path, model=base_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    @staticmethod
    def _prepare_full_model(model, classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes, activation='softmax')(flatten_in)

        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )   
        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model=self._prepare_full_model(
            model=self.get_base_model(),
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)


        
        

In [ ]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e